In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu
Created on Jan. 14, 2024
Last modified on Feb. 4, 2024

Convert the stellar yield file from Sukhbold et al. (2016) to the format more convenient for 
further IMF determination.

Note that the yields from Sukhbold et al. (2016) are total yields.
We can get the net yields by minusing the progenitor model.

You can download the yields table and progenitor models from
https://wwwmpa.mpa-garching.mpg.de/ccsnarchive/data/SEWBJ_2015/index.html
"""
import numpy as np
import pandas as pd
import pyatomdb
import re
import os
from functools import reduce

In [2]:
def get_S16_single_yields(file):
    """
    Get the yileds from S16.
    
    Parameters
    ----------
    file : str
        The file name of the yield table.
    
    Returns
    -------
    yields : dataframe
        The dataframe of the yields.
        Its indice are "H, He, Li ...".
        Its column is "yield (ejecta+wind)", whose name is the initial mass.
    """
    # Get the number between 's' and '.yield_table', which is the initial mass.
    pattern = re.compile(r's(\d+(?:\.\d+)?).yield_table')
    match = pattern.search(file)
    m_ini = match.group(1)
    if 'implosions_W18' in file:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('wind', 'f8')])

        y = pd.DataFrame(y['wind'], index=y['isotope'], columns=[m_ini])
    else:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('ejecta', 'f8'), ('wind', 'f8')])[:284]
        #  [:284] is to remove the unstable isotopes
        # y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        # y = pd.DataFrame(y['ejecta'], index=y['isotope'], columns=[m_ini])
    y.index = y.index.str.capitalize()
    y['element'] = y.index.str.extract(r'([A-Za-z]+)').to_numpy().ravel()  # The element symbol
    y['A'] = y.index.str.extract(r'(\d+)').to_numpy().ravel()  # The mass number
    y[m_ini] = y[m_ini]/y['A'].astype(float)  # The yield in number
    y = y.groupby('element')[m_ini].sum().reset_index()
    y['Z'] = y['element'].apply(pyatomdb.atomic.elsymb_to_Z)
    y.sort_values(by='Z', inplace=True)
    y.set_index('element', inplace=True)
    y.drop('Z', axis=1, inplace=True)
    y.reset_index(inplace=True)
    return y


def merge_two_dfs(left, right):
    """
    Define a function to merge two DataFrames
    """
    return pd.merge(left, right, how='outer')

# From Tuguldur Sukhbold:

This being said, I can address your specific question - when considering different engines we have segregated the stars into two categories:

- $M_{\mathrm{ZAMS}}\leq 12 M_{\mathrm{\odot}}$: we considered them as "Crab like"
- $M_{\mathrm{ZAMS}}> 12 M_{\mathrm{\odot}}$: we considered them as "87A like"

for stars with $\leq 12 M_{\mathrm{\odot}}$ we only applied the Z9.6 engine, and for heavier stars we applied 4 different engines (W18, N20, S19.8 etc...) to get the explosion outcomes. And when computing integrated yields we combine Z9.6 results for lighter stars with W18 or N20 engine results of heavier stars, e.g.:

- combination 1: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + W18 (for $> 12 M_{\mathrm{\odot}}$)
- combination 2: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + N20 (for $> 12 M_{\mathrm{\odot}}$)

In [3]:
S16mods = ["W18", "N20"]  # The yields in Z9.6 have been copied and pasted to the W18 and N20 folder.
for S16mod in S16mods:
    files = os.listdir('./Original/S16/%s/' % S16mod)
    files = [item for item in files if item.endswith('.yield_table')]
    tables = [ ]
    for file in files:
        tables.append(get_S16_single_yields('./Original/S16/%s/%s' % (S16mod, file)))
    files = os.listdir('./Original/S16/Z9.6/')
    files = [item for item in files if item.endswith('.yield_table')]
    for file in files:
        tables.append(get_S16_single_yields('./Original/S16/%s/%s' % ('Z9.6', file)))
    merged_df = reduce(merge_two_dfs, tables)
    merged_df.set_index('element', inplace=True) 
    merged_df.reset_index(inplace=True)
    cols = merged_df.columns.drop('element')
    cols1 = np.array([float(item) for item in cols])
    cols = cols[cols1.argsort()]
    cols = cols.insert(0, 'element')
    merged_df = merged_df[cols]
    merged_df.to_csv('./SNcc/S16-0.0200-SNcc-%s-total.csv' % S16mod, index=False, float_format='%.2e')
    # The float format of original yield table is '%.2e'.

In [25]:
progenitor_dir = './Original/S16/progenitor_models/'
files = np.array(os.listdir(progenitor_dir))
M_ini = np.array([float(re.search(r's(.*?)_presn', file).group(1)) for file in files])
files = files[M_ini.argsort()]

In [36]:
cols = ['grid', 'CellOuterTotalMass', 'CellOuterRadius', 'CellOuterVelocity', 'CellDensity', 
        'CellTemperature', 'CellPressure', 'CellSpecificEnergy', 'CellSpecificEntropy', 
        'CellSpecificAngularVelocity', 'CellA_bar', 'CellY_e', 'Stability', 'Network', 
        'Neutrons', 'H1', 'He3', 'He4', 'C12', 'N14', 'O16', 'Ne20', 'Mg24', 'Si28', 
        'S32', 'Ar36', 'Ca40', 'Ti44', 'Cr48', 'Fe52', 'Fe54', 'Ni56', 'Fe56', 'Fe']

for file in files[:1]:
    tmp = pd.read_table(progenitor_dir+file, sep=r'\s+', skiprows=2, header=None, 
                        names=cols)

In [40]:
tmp.iloc[:, tmp.columns.get_loc('H1'):tmp.columns.get_loc('Fe')]

,H1,He3,He4,C12,N14,O16,Ne20,Mg24,Si28,S32,Ar36,Ca40,Ti44,Cr48,Fe52,Fe54,Ni56,Fe56
0,8.464101e-08,2.128006e-10,0.000141,1.846308e-10,2.195402e-11,2.561607e-10,2.544486e-10,3.360622e-08,0.000002,0.000004,0.000042,0.031174,0.091963,0.149682,2.678802e-12,0.726868,7.869591e-16,---
1,8.491987e-08,2.103728e-10,0.000142,1.842529e-10,2.196929e-11,2.560542e-10,2.595555e-10,3.503264e-08,0.000002,0.000004,0.000043,0.031961,0.094426,0.152689,2.826435e-12,0.720609,8.351928e-16,---
2,8.537872e-08,2.093568e-10,0.000143,1.846252e-10,2.205909e-11,2.562717e-10,2.635113e-10,3.608955e-08,0.000002,0.000005,0.000044,0.032457,0.096102,0.154801,2.954671e-12,0.716324,8.783373e-16,---
3,8.601227e-08,2.084565e-10,0.000144,1.852869e-10,2.219724e-11,2.568225e-10,2.688039e-10,3.746876e-08,0.000002,0.000005,0.000045,0.033082,0.098182,0.157371,3.117318e-12,0.711046,9.333358e-16,---
4,8.662985e-08,2.060642e-10,0.000145,1.855012e-10,2.226185e-11,2.567389e-10,2.740778e-10,3.907658e-08,0.000002,0.000005,0.000046,0.033784,0.100742,0.160654,3.343828e-12,0.704498,1.011385e-15,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,6.763725e-01,4.115719e-05,0.308563,1.502579e-03,2.584968e-03,5.898765e-03,1.306882e-03,7.916153e-04,0.000829,0.000423,0.000113,0.000074,0.000004,0.000034,1.869252e-75,0.001461,0.000000e+00,---
1405,6.765023e-01,4.116510e-05,0.308433,1.502828e-03,2.583722e-03,5.899858e-03,1.306881e-03,7.916146e-04,0.000829,0.000423,0.000113,0.000074,0.000004,0.000034,1.753244e-75,0.001461,0.000000e+00,---
1406,6.766196e-01,4.117225e-05,0.308315,1.503054e-03,2.582597e-03,5.900845e-03,1.306880e-03,7.916141e-04,0.000829,0.000423,0.000113,0.000074,0.000004,0.000034,1.652701e-75,0.001461,0.000000e+00,---
1407,6.767443e-01,4.117984e-05,0.308190,1.503293e-03,2.581400e-03,5.901894e-03,1.306879e-03,7.916135e-04,0.000829,0.000423,0.000113,0.000074,0.000004,0.000034,1.550603e-75,0.001461,0.000000e+00,---


In [45]:
tmp.loc[:, 'Ni56'].dtype == float

True